In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import cv2
import tensorflow
import keras
from sklearn.model_selection import train_test_split
import pickle
import os
import pandas as pd
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
 

In [ ]:
path = "PROBAR4enf" # folder with all the class folders
labelFile = 'labels4enf.csv' # file with all names of classes
batch_size_val=32  # how many to process together
steps_per_epoch_val=245
epochs_val=20
imageDimesions = (26,21,3)
testRatio = 0.2    # if 1000 images split will 200 for testing
validationRatio = 0.2 # if 1000 images 20% of remaining 800 will be 160 for validation

In [ ]:
count = 0
images = []
classNo = []
myList = os.listdir(path)
print("Total Classes Detected:",len(myList))
noOfClasses=len(myList)
print("Importing Classes.....")
for x in range (0,len(myList)):
    myPicList = os.listdir(path+"/"+str(count))
    for y in myPicList:
        curImg = cv2.imread(path+"/"+str(count)+"/"+y)
        curImg = cv2.resize(curImg,(21,26))
        #curImg = cv2.cvtColor(curImg,cv2.COLOR_BGR2GRAY)
        images.append(curImg)
        classNo.append(count)
    print(count, end =" ")
    count +=1
print(" ")
images = np.array(images)
classNo = np.array(classNo)
clas= np.unique(classNo)
print(images.shape)
print(clas)

In [ ]:
y = np.array(classNo)# se crea una matriz "Y" a partir de la lista "labels"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, y, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validationRatio, random_state=13)
print(X_train.shape)
 
# X_train = ARRAY OF IMAGES TO TRAIN
# y_train = CORRESPONDING CLASS ID
 

In [ ]:
#Cambiar las etiquetas de codificación categórica a one-hot
train_Y_one_hot = to_categorical(y_train)
test_Y_one_hot = to_categorical(y_test)

#Mostrar el cambio de etiqueta de categoría usando codificación one-hot
print('Original label:', y_train[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

In [ ]:
numOfSamples =[]
for  x in range(0, noOfClasses):
    numOfSamples.append(len(np.where(y_train==x)[0]))
print(numOfSamples)

In [ ]:
#TO CHECK IF NUMBER OF IMAGES MATCHES TO NUMBER OF LABELS FOR EACH DATA SET
print("Data Shapes")
print("Train",end = "");print(X_train.shape,y_train.shape)
print("Validation",end = "");print(X_validation.shape,y_validation.shape)
print("Test",end = "");print(X_test.shape,y_test.shape)
assert(X_train.shape[0]==y_train.shape[0]), "The number of images in not equal to the number of lables in training set"
assert(X_validation.shape[0]==y_validation.shape[0]), "The number of images in not equal to the number of lables in validation set"
assert(X_test.shape[0]==y_test.shape[0]), "The number of images in not equal to the number of lables in test set"
#assert(X_train.shape[1:]==(imageDimesions))," The dimesions of the Training images are wrong "
#assert(X_validation.shape[1:]==(imageDimesions))," The dimesionas of the Validation images are wrong "
#assert(X_test.shape[1:]==(imageDimesions))," The dimesionas of the Test images are wrong"
 

In [ ]:
 #READ CSV FILE
data=pd.read_csv(labelFile)
print("data shape ",data.shape,type(data))

In [ ]:
#DISPLAY SOME SAMPLES IMAGES  OF ALL THE CLASSES
num_of_samples = []
cols = 5
num_classes = noOfClasses
fig, axs = plt.subplots(nrows=num_classes, ncols=cols, figsize=(5, 10))
fig.tight_layout()
for i in range(cols):
    for j,row in data.iterrows():
        x_selected = X_train[y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0, len(x_selected)- 1), :, :], cmap="gray")
        axs[j][i].axis("off")
        if i == 2:
            axs[j][i].set_title(str(j)+ "-"+row["Name"])
            num_of_samples.append(len(x_selected))

In [ ]:
#PREPROCESSING THE IMAGES
 
def preprocessing(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255            # TO NORMALIZE VALUES BETWEEN 0 AND 1 INSTEAD OF 0 TO 255
    return img
 
X_train=np.array(list(map(preprocessing,X_train)))  # TO IRETATE AND PREPROCESS ALL IMAGES
X_validation=np.array(list(map(preprocessing,X_validation)))
X_test=np.array(list(map(preprocessing,X_test)))
#cv2.imshow("GrayScale Images",X_train[random.randint(0,len(X_train)-1)]) # TO CHECK IF THE TRAINING IS DONE PROPERLY
#cv2.waitKey(0)
print(X_train.shape)

In [ ]:
#ADD A DEPTH OF 1
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_validation=X_validation.reshape(X_validation.shape[0],X_validation.shape[1],X_validation.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)
print(X_train.shape)

In [ ]:
#DISPLAY A BAR CHART SHOWING NO OF SAMPLES FOR EACH CATEGORY
print(num_of_samples)
plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the training dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")
plt.show()

In [ ]:
#AUGMENTATAION OF IMAGES: TO MAKEIT MORE GENERIC
dataGen= ImageDataGenerator(width_shift_range=0.1,   # 0.1 = 10%     IF MORE THAN 1 E.G 10 THEN IT REFFERS TO NO. OF  PIXELS EG 10 PIXELS
                            height_shift_range=0.1,
                            zoom_range=0.2,  # 0.2 MEANS CAN GO FROM 0.8 TO 1.2
                            shear_range=0.1,  # MAGNITUDE OF SHEAR ANGLE
                            rotation_range=10)  # DEGREES
dataGen.fit(X_train)
batches= dataGen.flow(X_train,y_train,batch_size=20)  # REQUESTING DATA GENRATOR TO GENERATE IMAGES  BATCH SIZE = NO. OF IMAGES CREAED EACH TIME ITS CALLED
X_batch,y_batch = next(batches)
  
y_train= to_categorical(y_train,noOfClasses)
y_validation = to_categorical(y_validation,noOfClasses)
y_test = to_categorical(y_test,noOfClasses)

In [ ]:
tomato_plants = Sequential()
tomato_plants.add(Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1, padding='same',input_shape=(21,28,1)))
tomato_plants.add(BatchNormalization())

tomato_plants.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
tomato_plants.add(BatchNormalization())
tomato_plants.add(Dropout(0.25))

tomato_plants.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
tomato_plants.add(MaxPooling2D(pool_size=(2, 2)))
tomato_plants.add(Dropout(0.25))

tomato_plants.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
tomato_plants.add(BatchNormalization())
tomato_plants.add(Dropout(0.25))

tomato_plants.add(Flatten())
tomato_plants.add(Dense(32, activation='linear'))
tomato_plants.add(LeakyReLU(alpha=0.2))
tomato_plants.add(BatchNormalization())
tomato_plants.add(Dropout(0.5))

tomato_plants.add(Dense(32, activation='linear'))
tomato_plants.add(LeakyReLU(alpha=0.2))
tomato_plants.add(BatchNormalization())
tomato_plants.add(Dropout(0.5))

tomato_plants.add(Dense(noOfClasses, activation='softmax'))

In [ ]:
tomato_plants.summary()

In [ ]:
tomato_plants.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [ ]:
#modelo listo para entrenar
#tomato_train = tomato_plants.fit(X_train, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))
#history=tomato_plants.fit_generator(dataGen.flow(X_train,y_train,batch_size=batch_size_val),steps_per_epoch=steps_per_epoch_val,epochs=epochs_val,validation_data=(X_validation,y_validation),shuffle=1)
#steps= 245
#train_Y_one_hot
#test_Y_one_hot 
history= tomato_plants.fit_generator(dataGen.flow(X_train,train_Y_one_hot,
                            batch_size=batch_size_val),
                            steps_per_epoch=steps_per_epoch_val,
                            epochs=epochs_val,
                            validation_data=(X_validation,y_validation),
                            shuffle=1)

In [ ]:
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('loss')
plt.xlabel('epoch')
plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','validation'])
plt.title('Acurracy')
plt.xlabel('epoch')
plt.show()
score =tomato_plants.evaluate(X_test,y_test,verbose=0)
print('Test Score:',score[0])
print('Test Accuracy:',score[1])

In [ ]:
# STORE THE MODEL AS A PICKLE OBJECT
pickle_out= open("A/model2.p","wb")  # wb = WRITE BYTE
pickle.dump(tomato_plants,pickle_out)
pickle_out.close()
cv2.waitKey(0)
#al ejecutar la parte de #store the model as a pickle object. Sale este error: TypeError: cannot pickle 'weakref' object. Ayuda por favor 

In [ ]:
#CONVOLUTION NEURAL NETWORK MODEL
def myModel():
    no_Of_Filters=60
    size_of_Filter=(5,5) # THIS IS THE KERNEL THAT MOVE AROUND THE IMAGE TO GET THE FEATURES.
                         # THIS WOULD REMOVE 2 PIXELS FROM EACH BORDER WHEN USING 32 32 IMAGE
    size_of_Filter2=(3,3)
    size_of_pool=(2,2)  # SCALE DOWN ALL FEATURE MAP TO GERNALIZE MORE, TO REDUCE OVERFITTING
    no_Of_Nodes = 500   # NO. OF NODES IN HIDDEN LAYERS
    tomato_plants= Sequential()
    tomato_plants.add((Conv2D(no_Of_Filters,size_of_Filter,input_shape=(26,21,1),activation='relu')))  # ADDING MORE CONVOLUTION LAYERS = LESS FEATURES BUT CAN CAUSE ACCURACY TO INCREASE
    tomato_plants.add((Conv2D(no_Of_Filters, size_of_Filter, activation='relu')))
    tomato_plants.add(MaxPooling2D(pool_size=size_of_pool)) # DOES NOT EFFECT THE DEPTH/NO OF FILTERS
 
    tomato_plants.add((Conv2D(no_Of_Filters//2, size_of_Filter2,activation='relu')))
    tomato_plants.add((Conv2D(no_Of_Filters // 2, size_of_Filter2, activation='relu')))
    tomato_plants.add(MaxPooling2D(pool_size=size_of_pool))
    tomato_plants.add(Dropout(0.5))
 
    tomato_plants.add(Flatten())
    tomato_plants.add(Dense(no_Of_Nodes,activation='relu'))
    tomato_plants.add(Dropout(0.5)) # INPUTS NODES TO DROP WITH EACH UPDATE 1 ALL 0 NONE
    tomato_plants.add(Dense(noOfClasses,activation='softmax')) # OUTPUT LAYER
    # COMPILE MODEL
    tomato_plants.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return tomato_plants

In [ ]:
tomato_plants = myModel()
print(tomato_plants.summary())

In [ ]:
tomato_plants = myModel()
history=tomato_plants.fit_generator(dataGen.flow(X_train,y_train,batch_size=batch_size_val),steps_per_epoch=steps_per_epoch_val,epochs=epochs_val,validation_data=(X_validation,y_validation),shuffle=1)

In [ ]:
# STORE THE MODEL AS A PICKLE OBJECT
pickle_out= open("A/model4enf_gray-generator2.p","wb")  # wb = WRITE BYTE
pickle.dump(tomato_plants,pickle_out)
pickle_out.close()
cv2.waitKey(0)

In [ ]:
history= tomato_plants.fit_generator(dataGen.flow(X_train,train_Y_one_hot,
                            batch_size=batch_size_val),
                            steps_per_epoch=steps_per_epoch_val,
                            epochs=epochs_val,
                            validation_data=(X_validation,y_validation),
                            shuffle=1)

In [ ]:
from skimage.transform import resize

images=[]
# AQUI ESPECIFICAMOS UNAS IMAGENES
img = ['septoria_leaf_spot4.JPG']

def preprocessing(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    #img = img/255            # TO NORMALIZE VALUES BETWEEN 0 AND 1 INSTEAD OF 0 TO 255
    return img


for filepath in filenames:
    image = plt.imread(filepath,0)
    image = preprocessing(img)
    image_resized = resize(image, (26, 21),anti_aliasing=True,clip=False,preserve_range=True)
    images.append(image_resized)

X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
test_X = X.astype('float32')
test_X = test_X / 255.

predicted_classes = tomato_plants.predict(test_X)

for i, img_tagged in enumerate(predicted_classes):
    print(filenames[i], labels[img_tagged.tolist().index(max(img_tagged))])